### Step 1: Mount the Google Drive

Remember to use GPU runtime before mounting your Google Drive. (Runtime --> Change runtime type).

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Step 2: Open the project directory

Replace `Your_Dir` with your own path.

In [25]:
# cd /content/drive/MyDrive/Education 🎓/UCLA 🐻/3rd Year/ECE C147/emg2qwerty-main
#cd /content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main
import os
os.chdir('/content/drive/Shared drives/UCLA/ECE_C147/emg2qwerty-main')

### Step 3: Install required packages

After installing them, Colab will require you to restart the session.

In [ ]:
!pip install -r requirements.txt
#pip install numpy==1.26
#!pip install numpy==1.23.5 tensorflow==2.12 jax==0.4.13 jaxlib==0.4.13

  Cloning https://github.com/stepanhruda/camel-kenlm.git (to revision a6e906b2b30497dd999cb9c84c42c5111f8616e0) to /tmp/pip-req-build-3xnrktuh
  Running command git clone --filter=blob:none --quiet https://github.com/stepanhruda/camel-kenlm.git /tmp/pip-req-build-3xnrktuh
  Running command git rev-parse -q --verify 'sha^a6e906b2b30497dd999cb9c84c42c5111f8616e0'
  Running command git fetch -q https://github.com/stepanhruda/camel-kenlm.git a6e906b2b30497dd999cb9c84c42c5111f8616e0
  Running command git checkout -q a6e906b2b30497dd999cb9c84c42c5111f8616e0
  Resolved https://github.com/stepanhruda/camel-kenlm.git to commit a6e906b2b30497dd999cb9c84c42c5111f8616e0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Step 4: Start your experiments!

- Remember to download and copy the dataset to this directory: `Your_Dir/emg2qwerty/data`.
- You may now start your experiments with any scripts! Below are examples of single-user training and testing (greedy decoding).
- **There are two ways to track the logs:**
  - 1. Keep `--multirun`, and the logs will not be printed here, but they will be saved in the folder `logs`, e.g., `logs/2025-02-09/18-24-15/submitit_logs/`.
  - 2. Comment out `--multirun` and the logs will be printed in this notebook, but they will not be saved.

#### Training

- The checkpoints are saved in the folder `logs`, e.g., `logs/2025-02-09/18-24-15/checkpoints/`.

In [ ]:
# Single-user training
!python -m emg2qwerty.train \
  user="single_user" \
  trainer.accelerator=gpu trainer.devices=1 trainer.max_epochs=30\
  --multirun

2025-03-12 17:54:57.920485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741802098.169900    6407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741802098.238193    6407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 17:54:58.784968: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[2025-03-12 17:55:13,529][HYDRA] Submitit 'local' sweep output dir : logs/2025-03-12/17-55-13
[2025-03-12 17:55:13,53

#### Testing:

- Replace `Your_Path_to_Checkpoint` with your checkpoint path.

In [ ]:
# Single-user testing
!python -m emg2qwerty.train \
  user="single_user" \
  checkpoint="'/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/18-21-57/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/checkpoints/epoch=29-step=3600.ckpt'" \
  train=False trainer.accelerator=gpu \
  decoder=ctc_greedy \
  hydra.launcher.mem_gb=64 \
  --multirun

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/emg2qwerty/train.py", line 15, in <module>
    import pytorch_lightning as pl
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/__init__.py", line 35, in <module>
    from pytorch_lightning.callbacks import Callback  # noqa: E402
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/__init__.py", line 14, in <module>
    from pytorch_lightning.callbacks.batch_size_finder import BatchSizeFinder
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/batch_size_finder.py", line 24, in <module>
    from pytorch_lightning.callbacks.callback import Callback
  File "/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/callback.py", line 25, in <module>
    from pyto

In [ ]:
%load_ext tensorboard

baseline_train = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-03/20-15-30/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_train = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-09/04-13-49/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
GRU_train = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-09/05-57-08/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
CNNRNN_train = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-09/07-04-11/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_modified_train_128 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/02-03-42/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_modified_train_64 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/03-13-57/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_modified_train_256 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/04-24-00/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_modified_train_512 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/05-59-37/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_modified_train_512_6_3 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/18-21-57/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"
LSTM_modified_train_512_channel8 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-12/03-47-05/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"

baseline_test = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-10/23-57-27/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
LSTM_test = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/00-25-30/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
GRU_test = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/00-27-14/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
CNNRNN_test = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/00-29-43/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
LSTM_modified_test_128 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/03-11-29/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
LSTM_modified_test_64 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/04-17-44/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
LSTM_modified_test_256 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/05-33-59/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
LSTM_modified_test_512 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/07-41-20/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"
LSTM_modified_test_512_6_3 = "/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-11/20-30-17/job0_decoder=ctc_greedy,train=False,user=single_user/lightning_logs"

%tensorboard --logdir="/content/drive/Shareddrives/UCLA/ECE_C147/emg2qwerty-main/logs/2025-03-03/20-15-30/job0_trainer.devices=1,trainer.max_epochs=30,user=single_user/lightning_logs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 826), started 0:43:34 ago. (Use '!kill 826' to kill it.)

<IPython.core.display.Javascript object>